# Imports

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from deap import creator, base, tools, algorithms

# Dataset

In [41]:
# Load dataset 
# header=None because the first row is not a header
data = pd.read_excel('Project3_DistancesMatrix.xlsx', header=None) 

# Verified if the dataset has missing values
print(data.isnull().sum().sum())

# Used to create the distances.csv file for testing, not necessary for solving the problem
"""  
# from np.array() to dataframe pandas
distances123 = pd.DataFrame(distances)

# distances to csv
distances123.to_csv('distances.csv', index=False, header=False)

# Load dataset with float values to pandas dataframe
distances_test = pd.read_csv('distances.csv', header=None)

# pd to numpy array
distances_test = distances_test.to_numpy() 
"""

0


# Evolutionary Algorithm 

In [33]:
# Exclude first row and column, beacuse they are labels
distances = data.values[1:, 1:]  

def evalTSP(individual: list) -> tuple:
    """Fitness of an individual. The fitness is the total distance of the route.

    Args:
        individual (list): Individual to evaluate

    Returns:
        tuple: Tuple with a float value of Fitness of the individual
    """    

    # Add the Central at the start and the end of the route
    route = [0] + individual + [0]
    total_distance = sum(distances[route[i-1]-1][route[i]-1] for i in range(len(route)))
    
    # Return a tuple with a single value because is a requsiite of DEAP
    return (total_distance,)  

In [29]:
# Genetic algorithm parameters

# FitnessMin is a class inherited from base.Fitness and it has weights and values attributes
# weights = (-1.0,) because we want to minimize the total distance, (1.0) if we want to maximize
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# Individual is a list of integers (ecopoints)
creator.create("Individual", list, fitness=creator.FitnessMin)

# Toolbox is a container for the tools that will be used
toolbox = base.Toolbox()

# 100 ecopoints, in this case
number_of_ecopoints = len(data) - 1

# Start from 0 as 0 is Central
# Register genetic operators: mate, mutate, select
# toolbox.registetr("name of var", function, parameters of the function, ...)
toolbox.register("indices", random.sample, range(number_of_ecopoints), number_of_ecopoints) 
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.indices)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Register genetic operators
# mate: cxOrdered - order crossover
# mutate: mutShuffleIndexes - shuffle mutation
# select: selTournament - tournament selection
toolbox.register("mate", tools.cxOrdered)
toolbox.register("mutate", tools.mutShuffleIndexes, indpb=0.05)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evalTSP)

c:\Users\leo_b\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
c:\Users\leo_b\anaconda3\lib\site-packages\deap\creator.py:138: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [30]:
random.seed(169)

# Population
pop = toolbox.population(n=100)

# Hall of fame of 1 best individual
hof = tools.HallOfFame(1)

stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)

print("Start of evolution")
#algorithms.eaSimple(pop, toolbox, cxpb, mutpb, ngen, stats=None, halloffame=None)
# cxpb: probability of mating two individuals
# mutpb: probability of mutating an individual
# ngen: number of generations
algorithms.eaSimple(pop, toolbox, 0.7, 0.2, 1000, stats=stats, halloffame=hof)

# Best individual
print(f"Best individual is: {hof[0]} \nwith fitness: {hof[0].fitness}")



Start of evolution
gen	nevals	avg    	min  	max  
0  	100   	338.026	290.6	381.5
1  	80    	324.195	273.6	361.8
2  	84    	315.669	280.7	359.9
3  	86    	312.226	279.6	348.3
4  	71    	307.77 	274.5	343.3
5  	76    	304.339	279.6	337.1
6  	75    	300.378	279.6	331.5
7  	75    	298.496	275.9	337.4
8  	75    	295.318	268.1	340.1
9  	74    	289.877	269.1	329.8
10 	69    	284.364	254.3	317.8
11 	68    	281.264	252  	312  
12 	82    	277.098	251.6	310.1
13 	74    	273.89 	247  	311.9
14 	76    	267.225	240.8	294.7
15 	66    	261.981	233.1	297.5
16 	73    	255.852	232.6	292.9
17 	74    	251.997	229.7	302.6
18 	64    	248.116	225.8	278.1
19 	70    	244.719	226.6	310.2
20 	70    	244.298	221.9	284.3
21 	86    	240.473	213  	279.7
22 	80    	239.226	219  	292.1
23 	67    	235.552	215.5	285.4
24 	75    	234.563	215.5	275.6
25 	82    	235.094	215.5	277.4
26 	85    	234.777	213.8	285.9
27 	64    	232.536	213.8	305.1
28 	84    	228.946	213.8	276.2
29 	72    	227.446	208.1	281.3
30 	75    	227.752	2

In [ ]:
# Plot average by number of iterations
plt.figure(figsize=(11, 4))
plt.plot(stats.select("avg"), label="Average")
plt.plot(stats.select("min"), label="Minimum")

# Best on so far

In [ ]:
best_individual = hof[0]
print(f"Best individual is: {best_individual} \nwith total distance: {best_individual.fitness.values[0]}")
print("Distances between the points: ")
for i in range(len(best_individual) - 1):
    print(f"Distance from {best_individual[i]} to {best_individual[i+1]}: {best_individual.distances[i]}")


IndexError: list index out of range

# Show example results for differently sized inputs (including run time).

# What is the length of the shortest route that runs through all 100 Ecopoints?


# Optional Report for Bonus points
By submitting a pdf containing the code you developed and the results you obtained in
section 5, you will get a 1 val bonus on Project 3 (available soon). The optional report
should be submitted together with Project 3. 